In [30]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, uniform

In [31]:
theta_true = 3

In [32]:
mu_prior = 5 # Mean of gaussian prior
sigma_prior = 3 # Variance of gaussian prior

prior = norm(mu_prior, sigma_prior)
sigma_epsilon = 0.2 # Variance of observation noise
noise = norm(0, sigma_epsilon)
true_posterior = norm()

class h_kernel(object):
    def __init__(self, A=5):
        self.A = A
    def __call__(self, theta):
        return self.A * theta


h = h_kernel(A=5)

sigma_epsilon = 0.2 # Variance of observation noise
def y_obv(theta, h=h, noise=noise):
    obv = h(theta) + noise.rvs(1)
    return obv[0]



In [33]:

def sigma_new(sigma_old, mu_old, sigma_epsilon=sigma_epsilon, h=h):
    J = h.A
    left_term = 1 / sigma_old
    right_term = ((J**2) * mu_old) / sigma_epsilon

    return 1 / (left_term + right_term)

def mu_new(sigma_current, mu_old, y, sigma_epsilon=sigma_epsilon, 
           h=h, mu_prior=mu_prior, sigma_prior=sigma_prior):
    J = h.A
    left_term = ((J * mu_old) * (y - h(mu_old))) / sigma_epsilon
    right_term = (mu_prior - mu_old) / sigma_prior
    
    return mu_old + sigma_current
